# Description + Algorithm

---

**This is  AAQAD : Alexu Arabic Question-Answer Dataset**

**Data collection procedure is almost same as SQUAD 2.0 dataset**

**The main objective of this Dataset is to answer MRQA Problem in Arabic**

**V2.0 contains major real-time bug fixes**

---

                                            ALGORITHM  
            
            For each article in SQUAD 2.0:

                 Open article’s wikipedia English page.

                 If an Arabic version of this page exists:

                            Translate English page using Google Translate

                            Find matching translated paragraphs with the Arabic page

                            For each matched paragraph in each article:

                                      Save this paragraph (Arabic version from Arabic Wikipedia page)

                                      For each Question in SQUAD 2.0 on this paragraph:

                                                  Translate it with its answer(s) using Google Translate

                                                  Save it with the corresponding paragraph in JSON format

                 Else:
                            Abort this article (will be not included in AAQAD )




# Imports + Packages install

In [0]:
!pip install googletrans

!pip install munkres


# !pip instal+l git+https://github.com/BoseCorp/py-googletrans.git --upgrade

# !pip3 install fake-useragent
  

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15776 sha256=205cf65df57d722a499192ae30f3b692a07345fb72fd8f096271c6efb2629b94
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
from google.colab import files
import requests
from bs4 import BeautifulSoup
import unicodedata
from googletrans import Translator
import difflib
import json
import re
import numpy as np
from nltk import ngrams
from nltk import TreebankWordTokenizer
from nltk import WordPunctTokenizer
from nltk import WhitespaceTokenizer
from textblob.base import BaseTokenizer
import gensim
from scipy import spatial
from nltk import ngrams
import operator
from munkres import Munkres, print_matrix
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
# from fake_useragent import UserAgent
import random
import requests
import time

# Implementation Functions

## **Retrieve Arabic's Wikipedia paragarphs (if exists)**

### **Code**

In [0]:
##Given article title from SQUAD 2.0
#Check if arabic page exists and retrieve its arabic paragraphs

#returns a boolean for arabic page existance
#and the arabic title
#and the arabic paragraphs

def get_arabic_paragraphs(title):
  
  arabic_page_exists = True
  
  html = requests.get('https://ar.wikipedia.org/wiki/'+title).text
  soup = BeautifulSoup(html, "html.parser")
  ar_paragraphs = [p.get_text() for p in soup.find_all("p")]
  ar_title = ''
   
  #check if arabic wikipedia page do not exists for a given title
  if len(ar_paragraphs) == 1 and "هذه الصفحة خالية" in ar_paragraphs[0]:
    
    arabic_page_exists = False  
   
  #if arabic wikepidia page exists 
  else:
    
    #get arabic title 
    ar_title = [h1.get_text() for h1 in soup.find_all("h1")]
    ar_title =  ar_title[0]
    
    #reformat arabic paragraphs
    for i in range(len(ar_paragraphs)):
#       ar_paragraphs[i] = unicodedata.normalize("NFD", ar_paragraphs[i])
      ar_paragraphs[i] = re.sub(r'(\[(\d+)\])|(\[بحاجة لمصدر\])', '', ar_paragraphs[i])
            
  return arabic_page_exists, ar_title, ar_paragraphs

### **Testing**

In [0]:
#testing

#check, ar_parag =  get_arabic_paragraphs("Wikipedia:Articles_in_many_other_languages_but_not_on_English_Wikipedia")

check, ar_title, ar_parag = get_arabic_paragraphs("Beyoncé")

print(ar_title)
# print(len(ar_parag))

# for p in ar_parag:
#   print(p)
#   print("##########################################")

# print("##########################################")
# for p in en_parag:
#   print(p)
#   print("##########################################")

بيونسيه


## Translate Arabic paragraphs

### **Code**

In [0]:
## dictionary to convert arabic digits to english

eastern_to_western = {"٠":"0","١":"1","٢":"2","٣":"3","٤":"4","٥":"5","٦":"6","٧":"7","٨":"8","٩":"9",
                      "0":"0","1":"1","2":"2","3":"3","4":"4","5":"5","6":"6","7":"7","8":"8","9":"9",
                      '²':'','₂':''}

In [0]:
## translate a given paragraph to arabic

def translate_to_arabic(paragraph):
  
  #delay 2 secs
  time.sleep(2)
  
  translator = Translator()
  translatedParagraph = translator.translate(paragraph, dest='ar')
  
  #replace arabic digits with english ones
  translatedParagraph = list(translatedParagraph.text)
   
  for i in range(0,len(translatedParagraph)):
    if translatedParagraph[i].isdigit():
      translatedParagraph[i] = eastern_to_western[translatedParagraph[i]] 

  translatedParagraph = "".join(translatedParagraph)
  
#   translatedParagraph = unicodedata.normalize("NFD", translatedParagraph)
  
  return translatedParagraph

### **Testing**

In [0]:
## testing

p = "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say; born September 4, 1981)[4] is an American singer, songwriter and actress. Born and raised in Houston, Texas, Beyoncé performed in various singing and dancing competitions as a child. She rose to fame in the late 1990s as lead singer of the R&B girl-group Destiny's Child, one of the best-selling girl groups in history. Their hiatus saw the release of her first solo album, Dangerously in Love (2003), which debuted at number one on the US Billboard 200 chart and earned her five Grammy Awards.[5] The album also featured the US Billboard Hot 100 number-one singles 'Crazy in Love' and 'Baby Boy'."

# p = "Beyoncé Giselle Knowl 20"

translate_to_arabic(p)


'بيونسي جيزيل نولز كارتر (/ biːˈjɒnseɪ / bee-YON- say ؛ من مواليد 4 سبتمبر 1981) [4] مغنية وكاتب أغاني وممثلة أمريكية. ولدت ونشأت في هيوستن ، تكساس ، بيونسيه في مختلف مسابقات الغناء والرقص كطفل. صعدت إلى الشهرة في أواخر التسعينيات كمغنية رئيسية لمجموعة ديستينيز تشايلد للبنات ، إحدى أفضل مجموعات الفتيات مبيعًا في التاريخ. شهدت الفجوة الخاصة بهم إصدار ألبومها الفردي الأول ، Dangerious in Love (2003) ، الذي ظهر لأول مرة في المرتبة الأولى على مخطط Billboard 200 الأمريكي وحاز على جوائز Grammy الخمس. [5] وضم الألبوم أيضًا أغنية US Billboard Hot 100 الفردية الأولى "Crazy in Love" و "Baby Boy".'

In [0]:
## testing arabic to english digit conversion

s = " حكم الدولة العثمانية من سنة ١١١١ حتى سنة ٣٣٣٣"
s = list(s)

for i in range(0,len(s)):
  if s[i].isdigit():
    s[i] = ''.join([eastern_to_western[c] for c in s[i]])
    
s = "".join(s)

print(s)

 حكم الدولة العثمانية من سنة 1111 حتى سنة 3333


In [0]:
## Issue  5

p1 = "There are also some extremely complex characters which have understandably become rather rare. According to Joël Bellassen (1989), the most complex Chinese character is /𪚥 (U+2A6A5) zhé  listen (help·info), meaning 'verbose' and containing sixty-four strokes; this character fell from use around the 5th century. It might be argued, however, that while containing the most strokes, it is not necessarily the most complex character (in terms of difficulty), as it simply requires writing the same sixteen-stroke character 龍 lóng (lit. 'dragon') four times in the space for one. Another 64-stroke character is /𠔻 (U+2053B) zhèng composed of 興 xīng/xìng (lit. 'flourish') four times."

p2 = "There are also some extremely complex characters which have understandably become rather rare. According to Joël Bellassen (1989), the most complex Chinese character is (U+2A6A5) zhé listen (help·info), meaning 'verbose' and containing sixty-four strokes; this character fell from use around the 5th century. It might be argued, however, that while containing the most strokes, it is not necessarily the most complex character (in terms of difficulty), as it simply requires writing the same sixteen-stroke character 龍 lóng (lit. 'dragon') four times in the space for one. Another 64-stroke character is / (U+2053B) zhèng composed of  xīng/xìng (lit. 'flourish') four times."



p3 = "The term 'Sumerian' is the common name given to the ancient non-Semitic inhabitants of Mesopotamia, Sumer, by the Semitic Akkadians. The Sumerians referred to themselves as ùĝ saĝ gíg-ga (cuneiform: 𒌦 𒊕 𒈪 𒂵), phonetically /uŋ saŋ giga/, literally meaning 'the black-headed people', and to their land as ki-en-gi(-r) ('place' + 'lords' + 'noble'), meaning 'place of the noble lords'. The Akkadian word Shumer may represent the geographical name in dialect, but the phonological development leading to the Akkadian term šumerû is uncertain. Hebrew Shinar, Egyptian Sngr, and Hittite Šanhar(a), all referring to southern Mesopotamia, could be western variants of Shumer."

p4 = "The term 'Sumerian' is the common name given to the ancient non-Semitic inhabitants of Mesopotamia, Sumer, by the Semitic Akkadians. The Sumerians referred to themselves as ùĝ saĝ gíg-ga , phonetically /uŋ saŋ giga/, literally meaning 'the black-headed people', and to their land as ki-en-gi(-r) ('place' + 'lords' + 'noble'), meaning 'place of the noble lords'. The Akkadian word Shumer may represent the geographical name in dialect, but the phonological development leading to the Akkadian term šumerû is uncertain. Hebrew Shinar, Egyptian Sngr, and Hittite Šanhar(a), all referring to southern Mesopotamia, could be western variants of Shumer."


p5 = "Miṣr (IPA: [mi̠sˤr] or Egyptian Arabic pronunciation: [mesˤɾ]; Arabic: مِصر‎) is the Classical Quranic Arabic and modern official name of Egypt, while Maṣr (IPA: [mɑsˤɾ]; Egyptian Arabic: مَصر) is the local pronunciation in Egyptian Arabic. The name is of Semitic origin, directly cognate with other Semitic words for Egypt such as the Hebrew מִצְרַיִם (Mitzráyim). The oldest attestation of this name for Egypt is the Akkadian 𒆳 𒈪 𒄑 𒊒 KURmi-iṣ-ru miṣru, related to miṣru/miṣirru/miṣaru, meaning 'border' or 'frontier'."

p6 = "Miṣr (IPA: [mi̠sˤr] or Egyptian Arabic pronunciation: [mesˤɾ]; Arabic: مِصر‎) is the Classical Quranic Arabic and modern official name of Egypt, while Maṣr (IPA: [mɑsˤɾ]; Egyptian Arabic: مَصر) is the local pronunciation in Egyptian Arabic. The name is of Semitic origin, directly cognate with other Semitic words for Egypt such as the Hebrew מִצְרַיִם (Mitzráyim). The oldest attestation of this name for Egypt is the Akkadian KURmi-iṣ-ru miṣru, related to miṣru/miṣirru/miṣaru, meaning 'border' or 'frontier'."





# import re

# regex = re.compile('[^a-zA-Z ]')

# #First parameter is the replacement, second parameter is your input string
# p7 = regex.sub('', p1)
# p8 = regex.sub('', p3)
# p9 = regex.sub('', p5)


t = translate_to_arabic(p1)
print(t)

t = translate_to_arabic(p3)
print(t)

t = translate_to_arabic(p5)
print(t)



# import nltk
# nltk.download('words')


# words = set(nltk.corpus.words.words())

# p8 = " ".join(w for w in nltk.wordpunct_tokenize(p5) 
#          if w.lower() in words or not w.isalpha())


# t = translate_to_arabic(p6)
# print(t)

# t = translate_to_arabic(p7)
# print(t)

# t = translate_to_arabic(p8)
# print(t)




### **Testing New Translation Methods**


In [0]:
## From here we generate a random user agent
ua = UserAgent() 

## Retrieve_proxies (IPs)
def retrieve_proxies():
  
  ## proxies table, contains proxies [ip, port]
  proxies = [] 
  
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })
    
  return proxies

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)


# retrieve_proxies (IPs)
# retrieve_proxies()

In [0]:
### Translate an English Paragraph to Arabic
##  take the English parag and output the Arabic one
##  all digits are kept into English format (0,1,...)

## To avoid blocking our ip from many requests 
#  the ip is changed after # of requests 
# REQUESTS_LIMIT = 10

## Total # of requests is saved here
# REQUESTS = 0

def translate_to_arabic(paragraph):
  translatedParagraph = ''  
  
  # keep trying to translate until succesfully done
  while True: 
    try:
      # translating
      translator = Translator(proxies = {'http': proxy['ip']+':'+proxy['port']})
#       translator = Translator()
      translatedParagraph = translator.translate(paragraph, dest='ar')

      # replace arabic digits with english ones
      translatedParagraph = list(translatedParagraph.text)
      for i in range(0,len(translatedParagraph)):
        if translatedParagraph[i].isdigit():
          translatedParagraph[i] = eastern_to_western[translatedParagraph[i]] 
      translatedParagraph = "".join(translatedParagraph)
      break
    except:
      # If error, delete this proxy and find another one
      del proxies[proxy_index]
      if len(proxies) == 0:
        retrieve_proxies()
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]
  
  return translatedParagraph

In [0]:
import requests

for i in range (0,100):
   
  print("Request #",i)
  
  url = 'http://translate.google.com/translate_a/t'
  params = {
      "text": "Simple is better than complex", 
      "sl": "en", 
      "tl": "ar", 
      "client": "p"
  }

  p = requests.get(url, params=params).content.decode("utf-8")

  print("Translation: ",p)


In [0]:

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

# Main function
def main():
  count = 0
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  # Choose a random proxy
  proxy_index = random_proxy()
  proxy = proxies[proxy_index]

  for n in range(1, 1000):
    print("\nIteration #",n)
           
    # Every 10 requests, generate a new proxy
    if n % 10 == 0:
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

    # Make the call: translate request
    try:
      print("Translating...")
      translator = Translator(proxies = {'http': proxy['ip']+':'+proxy['port']})
      p = translator.translate("paragraph", dest='ar')
      print("Translation: ",p)
        
    except: # If error, delete this proxy and find another one
      print('Invaild proxy')
      count += 1
      del proxies[proxy_index]
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]
  print("Count: ",count)
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)

if __name__ == '__main__':
  main()


Iteration # 1
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 2
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 3
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 4
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 5
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 6
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 7
Translating...
Translation:  Translated(src=en, dest=ar, text=فقرة, pronunciation=faqira, extra_data="{'translat...")

Iteration # 8
Translating...
Translation:  Translated(src=en, dest=a

In [0]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

# Main function
def main():
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  # Choose a random proxy
  proxy_index = random_proxy()
  proxy = proxies[proxy_index]

  for n in range(1, 100):
    req = Request('http://icanhazip.com')
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    # Every 10 requests, generate a new proxy
    if n % 10 == 0:
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

    # Make the call
    try:
      my_ip = urlopen(req).read().decode('utf8')
      print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
      del proxies[proxy_index]
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)

if __name__ == '__main__':
  main()

#1: 125.26.7.85

Proxy 125.26.7.85:54888 deleted.
Proxy 103.50.154.4:61330 deleted.
Proxy 191.37.183.209:60139 deleted.
Proxy 134.119.205.246:8080 deleted.
Proxy 80.73.90.18:3128 deleted.
#7: 93.39.228.188

Proxy 93.39.228.188:3128 deleted.
#9: 77.108.67.40

Proxy 27.72.72.48:61612 deleted.
Proxy 109.196.15.142:49133 deleted.
Proxy 118.174.234.80:48879 deleted.
Proxy 103.194.248.166:32662 deleted.
Proxy 49.51.195.24:1080 deleted.
Proxy 36.89.183.241:40109 deleted.
Proxy 43.239.205.233:61796 deleted.
Proxy 185.122.44.218:36805 deleted.
Proxy 77.108.67.40:54805 deleted.
Proxy 119.82.252.1:41568 deleted.


## **Similarity Pretrained Model**

**This Source Code is taken from:**
1. https://github.com/bakrianoo/aravec
2. https://github.com/adhaamehab/textblob-ar

### **Download Pretrained Word Embedding Model**

In [0]:
### downloading a pretrained arabic word embedding model
!wget -qq https://bakrianoo.sfo2.digitaloceanspaces.com/aravec/full_grams_cbow_300_wiki.zip -P ./data

### unzip the pretrained model  
!unzip -qq data/full_grams_cbow_300_wiki.zip -d ./data

### **utilities.py Module**

In [0]:
# =========================
# ==== Helper Methods =====

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

def get_vec(n_model,dim, token):
    vec = np.zeros(dim)
    exist = False
    is_vec = False
    if token not in n_model.wv:
        _count = 0
        is_vec = True
        for w in token.split("_"):
            if w in n_model.wv:
                _count += 1
                vec += n_model.wv[w]
        if _count > 0:
            vec = vec / _count
            exist = True
    else:
        vec = n_model.wv[token]
        exist = True
    
    return vec,exist

def calc_vec(pos_tokens, neg_tokens, n_model, dim):
    vec = np.zeros(dim)
    for p in pos_tokens:
        vec += get_vec(n_model,dim,p)
    for n in neg_tokens:
        vec -= get_vec(n_model,dim,n)
    
    return vec   

## -- Retrieve all ngrams for a text in between a specific range
def get_all_ngrams(text, nrange=3):
    text = re.sub(r'[\,\.\;\(\)\[\]\_\+\#\@\!\?\؟\^]', ' ', text)
    tokens = [token for token in text.split(" ") if token.strip() != ""]
    ngs = []
    for n in range(2,nrange+1):
        ngs += [ng for ng in ngrams(tokens, n)]
    return ["_".join(ng) for ng in ngs if len(ng)>0 ]

## -- Retrieve all ngrams for a text in a specific n
def get_ngrams(text, n=2):
    text = re.sub(r'[\,\.\;\(\)\[\]\_\+\#\@\!\?\؟\^]', ' ', text)
    tokens = [token for token in text.split(" ") if token.strip() != ""]
    ngs = [ng for ng in ngrams(tokens, n)]
    return ["_".join(ng) for ng in ngs if len(ng)>0 ]

## -- filter the existed tokens in a specific model
def get_existed_tokens(tokens, n_model):
    return [tok for tok in tokens if tok in n_model.wv ]


### **tokenizer.py Module**

In [0]:
class NLTKTreebankWordTokenizer(BaseTokenizer):

    def tokenize(self, text):
        return TreebankWordTokenizer().tokenize(text)

class NLTKWordPunctTokenizer(BaseTokenizer):

    def tokenize(self, text):
        return WordPunctTokenizer().tokenize(text)


class NLTKWhitespaceTokenizer(BaseTokenizer):

    def tokenize(self, text):
        return WhitespaceTokenizer().tokenize(text)


### **similarity.py Module**

In [0]:

class TextSimilarity:

    def __init__(self):
        try:
            self.model = gensim.models.Word2Vec.load('data/full_grams_cbow_300_wiki.mdl')
        except FileNotFoundError:
            raise FileNotFoundError
            
    def avg_feature_vector(self, sentence, num_features=300):
        words = NLTKWordPunctTokenizer().tokenize(clean_str(sentence))
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            word_vect,exist = get_vec(n_model=self.model, dim=num_features, token=word)
            feature_vec = np.add(feature_vec, word_vect)
            if exist:
              n_words += 1
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

    def similarity(self, sentence1, sentence2):
        vec1, vec2 = self.avg_feature_vector(sentence1), self.avg_feature_vector(sentence2)
        return self.cosine_similarity(vec1, vec2)

    def cosine_similarity(self, vec1, vec2):
        return 1 - spatial.distance.cosine(vec1, vec2)

### **Build Model**

In [0]:
sim = TextSimilarity()
# takes around 12 second (macbook pro 2017) to load the pretrained word2vec

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### **Testing**

In [0]:
## testing similarity Issue

sent1 = u'بيونسي جيزيل نولز كارتر (/ biːˈjɒnseɪ / bee-YON-say) (من مواليد 4 سبتمبر 1981) هي مغنية أمريكية وكاتبة أغاني ومنتجة تسجيل وممثلة. ولدت ونشأت في هيوستن بتكساس ، وأدت عروضها في العديد من مسابقات الغناء والرقص وهي طفلة ، واشتهرت في أواخر التسعينات كمغنية رئيسية في مجموعة ديستينيز تشايلد للبنات. أصبحت المجموعة ، التي يديرها والدها ، ماثيو نولز ، واحدة من أكثر المجموعات مبيعًا في العالم على الإطلاق. شهد توقفهم إطلاق ألبوم بيونسي لأول مرة ، Dangerious in Love (2003) ، الذي أسسها كفنانة منفردة في جميع أنحاء العالم ، وحاز على خمس جوائز Grammy وضمها إلى أفضل 100 أغنية فردية من فئة Billboard Hot "Crazy in Love" و "Baby Boy" .'

sent2 = u'بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 "كريزي إن لوف" و"بيبي بوي".'

sent3 = u'بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 "كريزي إن لوف" و"بيبي بوي".'


# print("Sim 1-2: ",sim.similarity(sent1,sent2))

# print("Sim 1-3: ",sim.similarity(sent1,sent3))

# print("Sim 2-3: ",sim.similarity(sent2,sent3))

# print(len(sent2)," - ",len(sent3))

# print(sent2 == sent3)

# print(sorted(sent2) == sorted(sent3))

# set1 = set(sent2.split(' '))
# set2 = set(sent3.split(' '))
# print(set1 == set2)

for i in range (0,len(sent3)):
  if sent2[i] != sent3[i]:
    print("index: ",i," sent2: ",sent2[i]," sent3: ",sent3[i])

# [i for i in range(len(sent3)) if sent2[i] != sent3[i]]
#mn awel index 82 le7ad index 716

# print(sent2[82],sent2[83],sent2[84],sent2[85],sent2[86],sent2[87],sent2[88],sent2[89],sent2[90],sent2[91],sent2[92],sent2[93],sent2[94],sent2[95])
# print(sent3[82],sent3[83],sent3[84],sent3[85],sent3[86],sent3[87],sent3[88],sent3[89],sent3[90],sent3[91],sent3[92],sent3[93],sent3[94],sent3[95])

# s = difflib.SequenceMatcher(None, sent2, sent3)
# print(s.ratio())
# for block in s.get_matching_blocks():
#   print (block)


index:  82  sent2:  ا  sent3:  أ
index:  83  sent2:  ٔ  sent3:  ت
index:  84  sent2:  ت  sent3:   
index:  85  sent2:     sent3:  ف
index:  86  sent2:  ف  sent3:  ي
index:  87  sent2:  ي  sent3:   
index:  88  sent2:     sent3:  ه
index:  89  sent2:  ه  sent3:  ي
index:  90  sent2:  ي  sent3:  و
index:  91  sent2:  و  sent3:  س
index:  92  sent2:  س  sent3:  ت
index:  93  sent2:  ت  sent3:  ن
index:  94  sent2:  ن  sent3:   
index:  95  sent2:     sent3:  ب
index:  96  sent2:  ب  sent3:  و
index:  97  sent2:  و  sent3:  ل
index:  98  sent2:  ل  sent3:  ا
index:  99  sent2:  ا  sent3:  ي
index:  100  sent2:  ي  sent3:  ة
index:  101  sent2:  ة  sent3:   
index:  102  sent2:     sent3:  ت
index:  103  sent2:  ت  sent3:  ك
index:  104  sent2:  ك  sent3:  س
index:  105  sent2:  س  sent3:  ا
index:  106  sent2:  ا  sent3:  س
index:  107  sent2:  س  sent3:  ،
index:  108  sent2:  ،  sent3:   
index:  109  sent2:     sent3:  ه
index:  110  sent2:  ه  sent3:  ي
index:  111  sent2:  ي  sent3:  

In [0]:
## testing similarity

sent1 = u'الإرهابي الصالح هي رواية خيال سياسي للكاتبة دوريس ليسينج. ظهرت أول طبعة للرواية في سبتمبر من عام 1985 للناشرين جوناثان كيب في المملكة المتحدة وألفريد أ'

# sent2 = u'روايه الكاتبه دوريس ليسينج هي روايه خيال سياسي ظهرت في سبتمبر 1985 بعنوان الارهابي الصالح وتم نشرها عن طريق جوناثان كيب والفريد أ في انجلترا'

sent2 = u'الكوكبة هي مجموعة من النجوم التي تكون شكلا أو صورة، وهي تدل على المنطقة التي تظهر فيها مجموعة محدودة من النجوم. وقد قسم الاتحاد الفلكي الدولي في عام 1930 السماء إلى 88 كوكبة، وذلك لتوحيد أشكال الكوكبات وعددها بعد أن كانت تتخيلها كل من الحضارات القديمة بشكل مختلف.'


sim.similarity(sent1, sent2)

0.34852882960983866

In [0]:
## testing similarity

arabicanswer = translate_to_arabic("Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\".")
print(arabicanswer)

#sent1 = u'بيونسي جيزيل نولز كارتر (/ biːˈjɒnseɪ / bee-YON-say) (من مواليد 4 سبتمبر 1981) هي مغنية وكاتب أغاني ومنتجة تسجيل وممثلة أمريكية. ولدت ونشأت في هيوستن بتكساس ، وأدت عروضها في العديد من مسابقات الغناء والرقص وهي طفلة ، واشتهرت في أواخر التسعينات كمغنية رئيسية في مجموعة ديستينيز تشايلد للبنات. أصبحت المجموعة ، التي يديرها والدها ، ماثيو نولز ، واحدة من أكثر المجموعات مبيعًا في العالم على الإطلاق. شهدت فترة توقفهم إطلاق ألبوم بيونسي لأول مرة ، Dangerious in Love (2003) ، الذي أسسها كفنانة منفردة في جميع أنحاء العالم ، وحصلت على خمس جوائز Grammy وعرضت أغنية Billboard Hot 100 الفردية الأولى "Crazy in Love" و "Baby Boy" .'
# sent2 = u'روايه الكاتبه دوريس ليسينج هي روايه خيال سياسي ظهرت في سبتمبر 1985 بعنوان الارهابي الصالح وتم نشرها عن طريق جوناثان كيب والفريد أ في انجلترا'
sent1 = u'بيونسي جيزيل نولز كارتر (/ biːˈjɒnseɪ / bee-YON-say) (من مواليد 4 سبتمبر 1981) هي مغنية أمريكية وكاتبة أغاني ومنتجة تسجيل وممثلة. ولدت ونشأت في هيوستن بتكساس ، وأدت عروضها في العديد من مسابقات الغناء والرقص وهي طفلة ، واشتهرت في أواخر التسعينات كمغنية رئيسية في مجموعة ديستينيز تشايلد للبنات. أصبحت هذه المجموعة ، التي يديرها والدها ، ماثيو نولز ، واحدة من أفضل مجموعات الفتيات مبيعًا في العالم على الإطلاق. شهد توقفهم إطلاق ألبوم بيونسي لأول مرة ، Dangerious in Love (2003) ، الذي أسسها كفنانة منفردة في جميع أنحاء العالم ، وحاز على خمس جوائز Grammy وضمها إلى أفضل 100 أغنية فردية من فئة Billboard Hot "Crazy in Love" و "Baby Boy" .'
sent2 = u'بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 "كريزي إن لوف" و"بيبي بوي".'
sim.similarity(arabicanswer, sent2)

بيونسي جيزيل نولز كارتر (/ biːˈjɒnseɪ / bee-YON-say) (من مواليد 4 سبتمبر 1981) هي مغنية أمريكية وكاتبة أغاني ومنتجة تسجيل وممثلة. ولدت ونشأت في هيوستن بتكساس ، وأدت عروضها في العديد من مسابقات الغناء والرقص وهي طفلة ، واشتهرت في أواخر التسعينات كمغنية رئيسية في مجموعة ديستينيز تشايلد للبنات. أصبحت هذه المجموعة ، التي يديرها والدها ، ماثيو نولز ، واحدة من أفضل مجموعات الفتيات مبيعًا في العالم على الإطلاق. شهد توقفهم إطلاق ألبوم بيونسي لأول مرة ، Dangerious in Love (2003) ، الذي أسسها كفنانة منفردة في جميع أنحاء العالم ، وحاز على خمس جوائز Grammy وضمها إلى أفضل 100 أغنية فردية من فئة Billboard Hot "Crazy in Love" و "Baby Boy" .


0.9158868432452788

### **Difflib built-in Similarity Method**

In [0]:
def difflib_similarity(paragraph1, paragraph2):
  
  sequence = difflib.SequenceMatcher(a = paragraph1, b = paragraph2, autojunk= False)
  difference = sequence.ratio()
  
  return difference  

## **Retrieve Similar Arabic Paragraphs**

### **Code**

In [0]:
## Given a translated arabic paragraphs from English wikipedia page
# it is required to retieve the most similar araibc
#  paragraphs from all wikipedia arabic paragraphs

## Hungarian Assignment Algorithm implementation
#  using Munkres built-in implementation algorithm

## return a dictionray:
#  key: index indicating the same order of the original
#       english paragraph in SQUAD
#  value: The corresponding arabic paragraph

## Similarity Threshold
PARAG_SIM_THRESHOLD = 0.78


def get_similar_ar_paragraphs(en_paragraphs, wiki_ar_paragraphs):
  
  # translate en praragraphs using google translate
  translated_ar_paragraphs = []
  for p in en_paragraphs:
    translated_ar_paragraphs.append(translate_to_arabic(p))
  
  
  # Prepare 2D cost matrix:
  #  rows: translated ar paragraphs
  #  cols: wiki ar paragraphs
  rows = len(translated_ar_paragraphs)
  cols = len(wiki_ar_paragraphs)
  cost_matrix = [[0.0 for i in range(cols)] for j in range(rows)] 
  
  for i in range (0,rows):
    for j in range (0,cols):
      #subtract 1 from similarities to convert it to minimization problem
      similarity = sim.similarity(translated_ar_paragraphs[i], wiki_ar_paragraphs[j])
      cost_matrix[i][j] = (1.0 - similarity) 
  
  
#   print("BEFORE :: Cost Matrix Dimensions:")
#   print("rows: ",len(cost_matrix)," cols: ",len(cost_matrix[0]))
  
  
  ## Optimization Efficency:
  #  Decrease cost matrix size to allow faster computations
  #  Delete entire rows & cols lower than the threshold
  cost_matrix = np.array(cost_matrix)
  #rows
  #cost_matrix = cost_matrix[~np.all(cost_matrix > (1.0-PARAG_SIM_THRESHOLD), axis=1)]
  cost_matrix = np.nan_to_num(cost_matrix)
  cost_matrix = cost_matrix.tolist()
  
  
#   print("AFTER :: Cost Matrix Dimensions:")
#   print("rows: ",len(cost_matrix)," cols: ",len(cost_matrix[0]))
   
#   print("Final Matrix:")
#   print(cost_matrix)
    
  # apply Munkres' algorithm on cost_matrix
  results = {}
  m = Munkres()
  indexes = m.compute(cost_matrix)
  
#   print("Munkres' Computations Done!")
  
  for row, column in indexes:
      if (-1*(cost_matrix[row][column] - 1) ) >= PARAG_SIM_THRESHOLD:
        results[row] = wiki_ar_paragraphs[column]
#         print("row: ",row," col: ",column)        
          
#         print("English Parag:")
#         print(en_paragraphs[row])
        
#         print("Corresponding ar Parag:")
#         print(wiki_ar_paragraphs[column])
        
#         print("Similarity: ",-1*(cost_matrix[row][column]-1))
#         print('\n')
        
  return results

### **Testing**

In [0]:

en_trans_parg = []
en_trans_parg.append("Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits \"Déjà Vu\", \"Irreplaceable\", and \"Beautiful Liar\". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for \"Single Ladies (Put a Ring on It)\". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.")
en_trans_parg.append("A self-described \"modern-day feminist\", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award's history. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011. Time listed her among the 100 most influential people in the world in 2013 and 2014. Forbes magazine also listed her as the most powerful female musician of 2015.")
en_trans_parg.append("Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\".")
en_trans_parg.append("Beyoncé attended St. Mary's Elementary School in Fredericksburg, Texas, where she enrolled in dance classes. Her singing talent was discovered when dance instructor Darlette Johnson began humming a song and she finished it, able to hit the high-pitched notes. Beyoncé's interest in music and performing continued after winning a school talent show at age seven, singing John Lennon's \"Imagine\" to beat 15/16-year-olds. In fall of 1990, Beyoncé enrolled in Parker Elementary School, a music magnet school in Houston, where she would perform with the school's choir. She also attended the High School for the Performing and Visual Arts and later Alief Elsik High School. Beyoncé was also a member of the choir at St. John's United Methodist Church as a soloist for two years.")

wiki_ar_parg = []
wiki_ar_parg.append(u"بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 'كريزي إن لوف' و'بيبي بوي'.")
wiki_ar_parg.append(u"حصلت بيونسي على تعليمها الإبتدائي في مدرسة القديس ماري التي في فريدريكسبورغ (تكساس). وحصلت هناك أيضاً على دروس الرقص. وقد اكتشف دارلاتا جونسون مدرس الرقص لبيونسي موهبتها الغنائية عندما بدأت في همهمة تلحين أغنية بصوتها (إحدى الأغنيات)، وقد ضغط عليها حتى تكمل الأغنية. وقد فازت بيونسي في مسابقة للغناء وأداء الصوت عندما كانت في السابعة من عمرها، وباقي المشتركين في الخامسة عشر، والسادسة عشر. وقد قامت بغناء أغنية تخيل للمغن والشاعر وعازف جيتار فرقة البيتلز جون لينون. وهي أغنية تصف عالمًا مثاليًا تتحقق فيه المساواة والسلام بين بني البشر. الأغنية من كلمات جون لينون ومن إخراج فيل سبيكتور. وقد صدرت لأول مرة عام-1971 كجزء من ألبوم إيماجين. وتعد الأغنية واحدة من أفضل الأغاني وأكثرها شيوعًا، وفي عام 2004 حلت في المرتبة الثالثة على قائمة رولينغ ستون لأفضل خمسمائة أغنية في التاريخ. وفي خريف عام 1990، وتنضم بيونسي إلى مدرسة باركر الابتدائية للموسيقى في هيوستن، تكساس. وبعد ذلك ذهبت إلى مدرسة أليف السيك، الثانوية للفنون المسرحية والفنون البصرية. بالإضافة إلى قضائها سنتين كاملتين في مدرسة الكنيسة الميثودية المتحدة St. John's.")
wiki_ar_parg.append(u"ولدت بيونسي في 4 سبتمبر 1981 في هيوستن، تكساس، وكان والدها ماثيو نولز يعمل مدير مبيعات في زيروكس وهي شركة عالمية متخصصة في مجال إدارة المستندات، والتي تصنع وتبيع عدد من طابعات الألوان والأبيض والأسود، والأنظمة متعددة الوظائف، وآلات التصوير الضوئي، بالإضافة إلى خدمات استشارية ذات علاقة، أما والدتها تينا نولز فكانت صاحبة كوافير ومصففة شعر. بيونسيه يأتي من لقب أمها. ووالد بيونسي من أصول إفريقية، أما والدتها التي تسكن في لويزيانا فلها أصول إفريقية، وفرنسية، وهندية، وأيرلندية. وسولانج هي أخت بيونسي الصغيرة، وتعمل مغنية وهي من أقرباء جوزيف بروسارد حاكم أكاديا .")
wiki_ar_parg.append(u"بعد انفصال دستنيز تشايلد في 2005، أصدرت ألبومها الإستديو الثاني المنفرد بي دي (2006)، والذي تضمن الأغاني التي نجحت عالمياً 'أيربليسبل' و'بيوتفل لاير'. بيونسي معروفة في التمثيل أيضاً، مع أدائها الذي ترشح لجائزة جولدن جلوب في دريم قيرلز (2006). وأدوار بطولة أخرى في ذا بينك بانتر (2006) وأوبسيسد (2009). وتزوجت من مغني الراب جي-زي وتجسيدها للشخصية المغنية الراحلة إيتا جيمس في كاديلاك ريكوردز (2008) أثر على ألبومها الإستديو الثالث، آيم... ساشا فيرس (2008)، والذي شهد ولادة الشخصية البديلة ساشا فيرس وحصل الألبوم على ستة جوائز غرامي في 2010، بما في ذلك أغنية السنة لأغنية 'سنقل ليديز (بوت آ رينق أون إت)'. أخذت بيونسي إجازة من عالم الفن في عام 2010 وتولت إدارة حياتها المهنية، أصدرت ألبومها الإستديو الرابع 4 (2011) الذي كان يحتوي على نبرة ناضجة أكثر من ألبوماتها السابقة، واستكشفت موسيقى الفانك الذي كان في السبعينات مع هذا الألبوم، والبوب في الثمنينات والسول التسعينات، كما أن ألبوم '4' تسرب بشكل كامل قبل صدوره بشكل رسمي مما يجعل الألبوم أكثر ألبوم تم تحميله بشكل غير قانوني من قِبل فنانة أنثى، رغم تسريبه بيعت منه أكثر من 8 مليون نسخة عالمياً  ألبومها الإستديو الخامس، بيونسي (2013)، تلقى الألبوم على الكثير من المراجعات الإيجابية من نقاد الموسيقى وفضلوه على ألبوماتها السابقة بسبب نوع الألبوم المظلم، كما أن الألبوم تم صدوره بشكل مفاجئ وبيعت منه 828,773 نسخة عالمياً في أول ثلاث أيام من صدوره من دون أي ترويج مسبق.")
wiki_ar_parg.append("وتقابلت بيونسي وصديقة طفولتها كيلي رولاند مع لا تفيا روبرسون في انتخابات مجموعات التسلية والترفيه التي تكونت من تجمع فتيات في عمر الثمانية. وقاموا بإنشاء مجموعة مع ثلاث فتيات آخريات وأسسوها بعنوان فتاة التيمي وقاموا بالرقص وغناء الراب في نطاق مسابقة المواهب في هيوستن. وبعد الإطلاع على المجموعة قام آرنى فراجير منتج R&B بإحضار الفتيات إلى الإستوديو في شمال كاليفورنيا، وفي ذلك الوقت شاركوا في مسابقة (البحث عن نجم) أكبر مسابقة مواهب عُرضت على شاشة التلفاز. وقامت بيونسي بأداء أغنية في مسابقة فتاة التيمي ولكنها لم تفز. وفي عام 1995 غادر والد بيونسي إدارة المجموعة. ونتيجة لذلك فقد حدّ دخل عائلة بيونسي واضطر كل من والدها ووالدتها الانتقال إلى شقق منفصلة. وانخفض مستوى مجموعة ماثيو المبتكر إلى الترتيب الرابع واستمرت هذه المجموعة في الظهور على المسرح وحتى من قبل ظهور مجموعة الفتيات R&B المعروفين. وقامت هذه المجموعة بعقد إتفاقية مع إلكترا للتسجيلات في عام 1995 وبعد وقت لاحق من هذه الإتفاقية تم إنهاؤها من قبل الشركة، وأدى هذا الحدث إلى زيادة التوتر في العائلة وانفصل والديّ بيونسي. وفي الخامس من شهر أكتوبر عام 1995 قامت الشعبية الترفيهية صاحبة دويني ويغينز بعقد إتفاقية مع الفتيات. وفي عام 1996 بدأت المجموعة بتسجيل ألبومها الأول في إطار الإتفاقية والذي قام بإنتاجه شركة سوني للموسيقى. وتوحدت أسرة نولز من جديد وبعد فترة ما وقعت الفتيات إتفاقية جديدة مع كولومبيا للتسجيلات.")


dic = get_similar_ar_paragraphs(en_trans_parg,wiki_ar_parg)

for key,val in dic.items():
  print(key," -> ",val)

BEFORE :: Cost Matrix Dimensions:
rows:  4  cols:  5
AFTER :: Cost Matrix Dimensions:
rows:  4  cols:  5
Final Matrix:
[[0.15884834151052896, 0.2037559206306565, 0.43854707175499963, 0.08460211509051385, 0.21616692222753897], [0.18581260175199255, 0.1929157655138397, 0.3721258876086231, 0.1716410080357239, 0.2192753058224951], [0.07982776549118853, 0.16554468513639276, 0.28484251081409495, 0.20141901934999706, 0.2525956033320099], [0.2580312544390648, 0.12079696359496106, 0.38026294135853655, 0.3336596571662196, 0.2287048859414611]]
Munkres' Computations Done!
row:  0  col:  3
row:  1  col:  4
row:  2  col:  0
row:  3  col:  1
0  ->  بعد انفصال دستنيز تشايلد في 2005، أصدرت ألبومها الإستديو الثاني المنفرد بي دي (2006)، والذي تضمن الأغاني التي نجحت عالمياً 'أيربليسبل' و'بيوتفل لاير'. بيونسي معروفة في التمثيل أيضاً، مع أدائها الذي ترشح لجائزة جولدن جلوب في دريم قيرلز (2006). وأدوار بطولة أخرى في ذا بينك بانتر (2006) وأوبسيسد (2009). وتزوجت من مغني الراب جي-زي وتجسيدها للشخصية المغنية الرا

In [0]:
cost_matrix = [ [0, 2, 2],[0, 5, 1],[0, 10, 2],[0, 8, 4] ]
cost_matrix = np.array(cost_matrix)

print("Before: \n",cost_matrix)

cost_matrix = np.array(cost_matrix)

#rows
cost_matrix = cost_matrix[~np.all(cost_matrix <= 2, axis=1)]

#cols
# cost_matrix = cost_matrix[~np.all(cost_matrix == 0, axis=0)]

cost_matrix = cost_matrix.tolist()


print(len(cost_matrix),len(cost_matrix[0]))

print("after: \n",cost_matrix)

Before: 
 [[ 0  2  2]
 [ 0  5  1]
 [ 0 10  2]
 [ 0  8  4]]
3 3
after: 
 [[0, 5, 1], [0, 10, 2], [0, 8, 4]]


In [0]:
cost = np.array([[5, 9, 1],[10, 3, 2],[8, 7, 4]])

from scipy.optimize import linear_sum_assignment
row_ind, col_ind = linear_sum_assignment(cost)

print("rows: ",row_ind)
print("cols: ",col_ind)

print("sum: ",cost[row_ind, col_ind].sum())

rows:  [0 1 2]
cols:  [2 1 0]
sum:  12


### **Testing New Method**

In [0]:
def get_similar_ar_paragraphs_V2(en_paragraphs, wiki_ar_paragraphs):
  
  # translate en praragraphs using google translate
  translated_ar_paragraphs = []
  for p in en_paragraphs:
    translated_ar_paragraphs.append(translate_to_arabic(p))
  
  
  # Prepare 2D cost matrix:
  #  rows: translated ar paragraphs
  #  cols: wiki ar paragraphs
  rows = len(translated_ar_paragraphs)
  cols = len(wiki_ar_paragraphs)
  cost_matrix = [[0.0 for i in range(cols)] for j in range(rows)] 
  
  for i in range (0,rows):
    for j in range (0,cols):
      #subtract 1 from similarities to convert it to minimization problem
      similarity = sim.similarity(translated_ar_paragraphs[i], wiki_ar_paragraphs[j])
      cost_matrix[i][j] = (1.0 - similarity) 
  
#   print("AFTER :: 2D cost matrix")
#   print(cost_matrix)
  
#   print("Cost Matrix Dimensions:")
#   print("rows: ",len(cost_matrix)," cols: ",len(cost_matrix[0]))
  
  # apply Munkres' algorithm on cost_matrix
  results = {}
  row_ind, col_ind = linear_sum_assignment(cost)
  
#   print("Munkres' computation Done!")
  
  for row, column in zip(row_ind, col_ind):
      if (-1*(cost_matrix[row][column] - 1) ) >= PARAG_SIM_THRESHOLD:
        results[row] = wiki_ar_paragraphs[column]
        print("row: ",row," col: ",column)        
          
#         print("English Parag:")
#         print(en_paragraphs[row])
        
#         print("Corresponding ar Parag:")
#         print(wiki_ar_paragraphs[column])
        
#         print("Similarity: ",-1*(cost_matrix[row][column]-1))
#         print('\n')
        
  return results

In [0]:
en_trans_parg = []
en_trans_parg.append("Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits \"Déjà Vu\", \"Irreplaceable\", and \"Beautiful Liar\". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for \"Single Ladies (Put a Ring on It)\". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.")
en_trans_parg.append("A self-described \"modern-day feminist\", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award's history. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011. Time listed her among the 100 most influential people in the world in 2013 and 2014. Forbes magazine also listed her as the most powerful female musician of 2015.")
en_trans_parg.append("Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\".")
en_trans_parg.append("Beyoncé attended St. Mary's Elementary School in Fredericksburg, Texas, where she enrolled in dance classes. Her singing talent was discovered when dance instructor Darlette Johnson began humming a song and she finished it, able to hit the high-pitched notes. Beyoncé's interest in music and performing continued after winning a school talent show at age seven, singing John Lennon's \"Imagine\" to beat 15/16-year-olds. In fall of 1990, Beyoncé enrolled in Parker Elementary School, a music magnet school in Houston, where she would perform with the school's choir. She also attended the High School for the Performing and Visual Arts and later Alief Elsik High School. Beyoncé was also a member of the choir at St. John's United Methodist Church as a soloist for two years.")

wiki_ar_parg = []
wiki_ar_parg.append(u"بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 'كريزي إن لوف' و'بيبي بوي'.")
wiki_ar_parg.append(u"حصلت بيونسي على تعليمها الإبتدائي في مدرسة القديس ماري التي في فريدريكسبورغ (تكساس). وحصلت هناك أيضاً على دروس الرقص. وقد اكتشف دارلاتا جونسون مدرس الرقص لبيونسي موهبتها الغنائية عندما بدأت في همهمة تلحين أغنية بصوتها (إحدى الأغنيات)، وقد ضغط عليها حتى تكمل الأغنية. وقد فازت بيونسي في مسابقة للغناء وأداء الصوت عندما كانت في السابعة من عمرها، وباقي المشتركين في الخامسة عشر، والسادسة عشر. وقد قامت بغناء أغنية تخيل للمغن والشاعر وعازف جيتار فرقة البيتلز جون لينون. وهي أغنية تصف عالمًا مثاليًا تتحقق فيه المساواة والسلام بين بني البشر. الأغنية من كلمات جون لينون ومن إخراج فيل سبيكتور. وقد صدرت لأول مرة عام-1971 كجزء من ألبوم إيماجين. وتعد الأغنية واحدة من أفضل الأغاني وأكثرها شيوعًا، وفي عام 2004 حلت في المرتبة الثالثة على قائمة رولينغ ستون لأفضل خمسمائة أغنية في التاريخ. وفي خريف عام 1990، وتنضم بيونسي إلى مدرسة باركر الابتدائية للموسيقى في هيوستن، تكساس. وبعد ذلك ذهبت إلى مدرسة أليف السيك، الثانوية للفنون المسرحية والفنون البصرية. بالإضافة إلى قضائها سنتين كاملتين في مدرسة الكنيسة الميثودية المتحدة St. John's.")
wiki_ar_parg.append(u"ولدت بيونسي في 4 سبتمبر 1981 في هيوستن، تكساس، وكان والدها ماثيو نولز يعمل مدير مبيعات في زيروكس وهي شركة عالمية متخصصة في مجال إدارة المستندات، والتي تصنع وتبيع عدد من طابعات الألوان والأبيض والأسود، والأنظمة متعددة الوظائف، وآلات التصوير الضوئي، بالإضافة إلى خدمات استشارية ذات علاقة، أما والدتها تينا نولز فكانت صاحبة كوافير ومصففة شعر. بيونسيه يأتي من لقب أمها. ووالد بيونسي من أصول إفريقية، أما والدتها التي تسكن في لويزيانا فلها أصول إفريقية، وفرنسية، وهندية، وأيرلندية. وسولانج هي أخت بيونسي الصغيرة، وتعمل مغنية وهي من أقرباء جوزيف بروسارد حاكم أكاديا .")
wiki_ar_parg.append(u"بعد انفصال دستنيز تشايلد في 2005، أصدرت ألبومها الإستديو الثاني المنفرد بي دي (2006)، والذي تضمن الأغاني التي نجحت عالمياً 'أيربليسبل' و'بيوتفل لاير'. بيونسي معروفة في التمثيل أيضاً، مع أدائها الذي ترشح لجائزة جولدن جلوب في دريم قيرلز (2006). وأدوار بطولة أخرى في ذا بينك بانتر (2006) وأوبسيسد (2009). وتزوجت من مغني الراب جي-زي وتجسيدها للشخصية المغنية الراحلة إيتا جيمس في كاديلاك ريكوردز (2008) أثر على ألبومها الإستديو الثالث، آيم... ساشا فيرس (2008)، والذي شهد ولادة الشخصية البديلة ساشا فيرس وحصل الألبوم على ستة جوائز غرامي في 2010، بما في ذلك أغنية السنة لأغنية 'سنقل ليديز (بوت آ رينق أون إت)'. أخذت بيونسي إجازة من عالم الفن في عام 2010 وتولت إدارة حياتها المهنية، أصدرت ألبومها الإستديو الرابع 4 (2011) الذي كان يحتوي على نبرة ناضجة أكثر من ألبوماتها السابقة، واستكشفت موسيقى الفانك الذي كان في السبعينات مع هذا الألبوم، والبوب في الثمنينات والسول التسعينات، كما أن ألبوم '4' تسرب بشكل كامل قبل صدوره بشكل رسمي مما يجعل الألبوم أكثر ألبوم تم تحميله بشكل غير قانوني من قِبل فنانة أنثى، رغم تسريبه بيعت منه أكثر من 8 مليون نسخة عالمياً  ألبومها الإستديو الخامس، بيونسي (2013)، تلقى الألبوم على الكثير من المراجعات الإيجابية من نقاد الموسيقى وفضلوه على ألبوماتها السابقة بسبب نوع الألبوم المظلم، كما أن الألبوم تم صدوره بشكل مفاجئ وبيعت منه 828,773 نسخة عالمياً في أول ثلاث أيام من صدوره من دون أي ترويج مسبق.")
wiki_ar_parg.append("وتقابلت بيونسي وصديقة طفولتها كيلي رولاند مع لا تفيا روبرسون في انتخابات مجموعات التسلية والترفيه التي تكونت من تجمع فتيات في عمر الثمانية. وقاموا بإنشاء مجموعة مع ثلاث فتيات آخريات وأسسوها بعنوان فتاة التيمي وقاموا بالرقص وغناء الراب في نطاق مسابقة المواهب في هيوستن. وبعد الإطلاع على المجموعة قام آرنى فراجير منتج R&B بإحضار الفتيات إلى الإستوديو في شمال كاليفورنيا، وفي ذلك الوقت شاركوا في مسابقة (البحث عن نجم) أكبر مسابقة مواهب عُرضت على شاشة التلفاز. وقامت بيونسي بأداء أغنية في مسابقة فتاة التيمي ولكنها لم تفز. وفي عام 1995 غادر والد بيونسي إدارة المجموعة. ونتيجة لذلك فقد حدّ دخل عائلة بيونسي واضطر كل من والدها ووالدتها الانتقال إلى شقق منفصلة. وانخفض مستوى مجموعة ماثيو المبتكر إلى الترتيب الرابع واستمرت هذه المجموعة في الظهور على المسرح وحتى من قبل ظهور مجموعة الفتيات R&B المعروفين. وقامت هذه المجموعة بعقد إتفاقية مع إلكترا للتسجيلات في عام 1995 وبعد وقت لاحق من هذه الإتفاقية تم إنهاؤها من قبل الشركة، وأدى هذا الحدث إلى زيادة التوتر في العائلة وانفصل والديّ بيونسي. وفي الخامس من شهر أكتوبر عام 1995 قامت الشعبية الترفيهية صاحبة دويني ويغينز بعقد إتفاقية مع الفتيات. وفي عام 1996 بدأت المجموعة بتسجيل ألبومها الأول في إطار الإتفاقية والذي قام بإنتاجه شركة سوني للموسيقى. وتوحدت أسرة نولز من جديد وبعد فترة ما وقعت الفتيات إتفاقية جديدة مع كولومبيا للتسجيلات.")


dic = get_similar_ar_paragraphs_V2(en_trans_parg,wiki_ar_parg)

for key,val in dic.items():
  print(key," -> ",val)

row:  1  col:  1
row:  2  col:  0
1  ->  حصلت بيونسي على تعليمها الإبتدائي في مدرسة القديس ماري التي في فريدريكسبورغ (تكساس). وحصلت هناك أيضاً على دروس الرقص. وقد اكتشف دارلاتا جونسون مدرس الرقص لبيونسي موهبتها الغنائية عندما بدأت في همهمة تلحين أغنية بصوتها (إحدى الأغنيات)، وقد ضغط عليها حتى تكمل الأغنية. وقد فازت بيونسي في مسابقة للغناء وأداء الصوت عندما كانت في السابعة من عمرها، وباقي المشتركين في الخامسة عشر، والسادسة عشر. وقد قامت بغناء أغنية تخيل للمغن والشاعر وعازف جيتار فرقة البيتلز جون لينون. وهي أغنية تصف عالمًا مثاليًا تتحقق فيه المساواة والسلام بين بني البشر. الأغنية من كلمات جون لينون ومن إخراج فيل سبيكتور. وقد صدرت لأول مرة عام-1971 كجزء من ألبوم إيماجين. وتعد الأغنية واحدة من أفضل الأغاني وأكثرها شيوعًا، وفي عام 2004 حلت في المرتبة الثالثة على قائمة رولينغ ستون لأفضل خمسمائة أغنية في التاريخ. وفي خريف عام 1990، وتنضم بيونسي إلى مدرسة باركر الابتدائية للموسيقى في هيوستن، تكساس. وبعد ذلك ذهبت إلى مدرسة أليف السيك، الثانوية للفنون المسرحية والفنون البصرية. بالإضافة إلى قضائ

## **Finding correct answer from Arabic Paragraphs**

### **Code**

In [0]:
buck2uni = {"'": u"\u2019", # hamza-on-the-line
            "|": u"\u0622", # madda
            ">": u"\u0623", # hamza-on-'alif
            "&": u"\u0624", # hamza-on-waaw
            "<": u"\u0625", # hamza-under-'alif
            "}": u"\u0626", # hamza-on-yaa'
            "*": u"\u0630", # dhaal
            "$": u"\u0634", # shiin
            "_": u"\u0640", # taTwil
            "~": u"\u0651", # shaddah
            "`": u"\u0670", # dagger 'alif
            "{": u"\u0671", # waSla
            "A": u"\u0627",
            "a": u"\u064E",
            "B": u"\u0628",
            "b": u"\u0628",
            "C": u"\u0643",
            "c": u"\u062B",
            "D": u"\u062F",
            "d": u"\u062F",
            "E": u"\u0639",
            "e": u"\u0650",
            "F": u"\u0641",
            "f": u"\u0641",
            "G": u"\u062C",
            "g": u"\u062C",
            "H": u"\u062D",
            "h": u"\u0647",
            "I": u"\u0625",
            "i": u"\u0650",
            "J": u"\u0686",
            "j": u"\u0686",
            "K": u"\u0643",
            "k": u"\u0643",
            "L": u"\u0644",
            "l": u"\u0644",
            "M": u"\u0645",
            "m": u"\u0645",
            "N": u"\u0646",
            "n": u"\u0646",
            "O": u"\u0648",
            "o": u"\u0619",
            "P": u"\u0628",
            "p": u"\u0628",
            "Q": u"\u0642",
            "q": u"\u0642",
            "R": u"\u0631",
            "r": u"\u0631",
            "S": u"\u0635",
            "s": u"\u0633",
            "T": u"\u0637",
            "t": u"\u062A",
            "U": u"\u0653",
            "u": u"\u0653",
            "V": u"\u06A4",
            "v": u"\u06A4",
            "W": u"\u0648",
            "w": u"\u0648",
            "X": u"\u062E",
            "x": u"\u062E",
            "Y": u"\u0649",
            "y": u"\u064A",
            "Z": u"\u0638",
            "z": u"\u0632",
            "ch": u"\u0634"
            
}


In [0]:
def transString(string, reverse=1):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buck2uni.items():
        if not reverse:
            string = string.replace(v, k)
        else:
            string = string.replace(k, v)

    return string

In [0]:
### finding exact matching answer -if exists- given an arabic paragraph
## uses similarity function to compare translated answer with possible
## answer from the arabic paragraphs...

ANS_SIM_THRESHOLD = 0.6

def find_answer(ar_paragraph,translated_question,translated_answer,english_answer):
  
  ## required to return
  answer_exist = True
  correct_answer = ''
  answer_start_index = 0
  ar_paragraph = ar_paragraph.strip()   
  
  ## calculate similarity for each sentence in each paragraph
  #  with  the translated question and save them into dic
  sentence_dic = {}
  ar_sentences = ar_paragraph.split('.')
  #remove empty entries from array
  ar_sentences = list(filter(None, ar_sentences))
   
#   print("There is ",ar_sentences," ar sentences")
  
  for  similar_sentence in ar_sentences:
    simlarity = sim.similarity(translated_question,similar_sentence)
    if similar_sentence != '':
          sentence_dic[similar_sentence]= simlarity
  
#   print("dic length: ",len(sentence_dic))
  
  
  #retrieve max 3 similar sentences
  sorted_sentence = []
  iterations = 0
  if len(ar_sentences) >= 3:
    iterations = 3
  else:
    iterations = len(ar_sentences)
  
  for i in range(0,iterations):
      sorted_sentence.append(max(sentence_dic.items(), key=operator.itemgetter(1)))
      sentence_dic.pop(max(sentence_dic.items(), key=operator.itemgetter(1))[0], None)
      
  
#   for e in range(0 , len(sorted_sentence)):
#     print(sorted_sentence[e])
#     print("----")


  if translated_answer.isdigit() :
    ## if trnaslated answer is digit
    
    max_difflib = -1
    #loop on max 3 similar sentences
    for  q in sorted_sentence:  
      ## retrieve exact similar word from similar_sentences
      # q[0] -> accesing key (paragraph) from each array entry
      words = q[0].split(' ')
      for i in range(0,len(words)):
        for j in range(i,len(words)):
          temp_ans = ''
          num_words = j-i+1
          l = i 
          for k in range(0,num_words):
            temp_ans += words[l] + ' '
            l += 1
          #print(temp_ans)
          difflib_sim = difflib_similarity(temp_ans,translated_answer)
          if difflib_sim > max_difflib:
            max_difflib = difflib_sim
            correct_answer = temp_ans
            #print(siml)
    
    # retrieve answer start index
    correct_answer = correct_answer.strip() 
    answer_start_index = ar_paragraph.find(correct_answer)
    
    
    if max_difflib < 0.5:
      answer_exist = False
       
    
  else:
    ## if trnaslated answer is NOT digit
    sim_dic = {}
    
    max_sim_answer = -1
    for  q in sorted_sentence:
      ## retrieve exact similar word from similar_sentences
      words = q[0].split(' ')

      for i in range(0,len(words)):
        for j in range(i,len(words)):

          temp_ans = ''
          num_words = j-i+1
          l = i 
          for k in range(0,num_words):
            temp_ans += words[l] + ' '
            l += 1
#           print(temp_ans)
          siml = sim.similarity(translated_answer,temp_ans)
#           print(siml)
          if (temp_ans != ' ') & (temp_ans != ''):
            sim_dic[temp_ans]= siml
          if siml > max_sim_answer:
            max_sim_answer = siml
            correct_answer = temp_ans
    
    temp_correct_ans = correct_answer
    
    if max_sim_answer >= ANS_SIM_THRESHOLD:
      # compare highest 3 similarity answers with difflib
      sorted_x = []
      for i in range(0,3):
        sorted_x.append(max(sim_dic.items(), key=operator.itemgetter(1)))
        sim_dic.pop(max(sim_dic.items(), key=operator.itemgetter(1))[0], None)

      max_difflib = -1
      for e in sorted_x:

        if e[1] >= ANS_SIM_THRESHOLD:
          difflib_sim = difflib_similarity(e[0],translated_answer)
          if difflib_sim > max_difflib:
            max_difflib = difflib_sim
            correct_answer = e[0]

      if max_difflib < 0.5:
        correct_answer = temp_correct_ans


      # retrieve answer start index
      correct_answer = correct_answer.strip() 
      answer_start_index = ar_paragraph.find(correct_answer)
    
    else:
    ## not a digit answer but less than threshold
    
      ## Issue: 5 -> English not translated answer in ar parags
      max_difflib = -1
      for  q in sorted_sentence:
        ## retrieve exact similar word from similar_sentences
        words = q[0].split(' ')
        for i in range(0,len(words)):
          for j in range(i,len(words)):
            temp_ans = ''
            num_words = j-i+1
            l = i 
            for k in range(0,num_words):
              temp_ans += words[l] + ' '
              l += 1
            #print(temp_ans)
            #compare with  ENgLISH answer !!
            difflib_sim = difflib_similarity(temp_ans,english_answer)
            #print(difflib_sim)
            if difflib_sim > max_difflib:
              max_difflib = difflib_sim
              correct_answer = temp_ans

      if max_difflib >= 0.5:
        # retrieve answer start index
        correct_answer = correct_answer.strip() 
        answer_start_index = ar_paragraph.find(correct_answer)

      else:
        ## Issue 6: literal transalation  ‘Destiny’s child’ -> ‘دستنى شايلد’	

        Transliterating = transString(english_answer)
        max_difflib = -1
        for  q in sorted_sentence:
          ## retrieve exact similar word from similar_sentences
          words = q[0].split(' ')
          for i in range(0,len(words)):
            for j in range(i,len(words)):
              temp_ans = ''
              num_words = j-i+1
              l = i 
              for k in range(0,num_words):
                temp_ans += words[l] + ' '
                l += 1
              #print(temp_ans)
              difflib_sim = difflib_similarity(temp_ans,Transliterating)
              #print(difflib_sim)
              if difflib_sim > max_difflib:
                max_difflib = difflib_sim
                correct_answer = temp_ans
                #print(siml)

        correct_answer = correct_answer.strip() 
        answer_start_index = ar_paragraph.find(correct_answer) 

        if max_difflib < 0.4:
  #         print("koukou")
#           print(max_difflib)
          answer_exist = False
        
            
  return  answer_exist, correct_answer, answer_start_index 

### **Testing**

In [0]:
par = u"بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول Dangerously in Love دانجيروسلي إن لوف (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 كريزي إن لوف و بيبي بوي"


ans = u"المعروفة باسم بيونسي"

# ans = ""


ans_exist, correct_ans, index = find_answer(par,ans)

print(ans_exist)
print(correct_ans)
print(index)


# print(par[52],par[53],par[54],par[55])


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


correct:  المعروفة باسم بيونسي 
True
المعروفة باسم بيونسي
52


In [0]:
x = {"adel": 0.22222222222222222222222222222222222, "sandra": 0.444444444444444444444444444444, "rimon": 0.3222222222222222222222222222222222,"hamada":1.0}

# sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse=True)

# for e in sorted_x:
#   print(e)

  
#   max(x.items(), key=operator.itemgetter(1))

('hamada', 1.0)
('sandra', 0.4444444444444444)
('rimon', 0.32222222222222224)


## Save data into JSON File

In [0]:
def save_JSON(data,count):
  
  with open('./data/AAQAD-v1.0.json', 'w') as outfile:
    json.dump(data, outfile)
  
#   # download the file on device after collecting each 5 parags
#   if (count != 0) & ((count % 5) == 0):
#     files.download('/content/data/AAQAD-v1.0.json')
  
  return

# Main Function [ Run Here ]

**To Avoid Cell inactivity Shutdown:**

So to prevent this just run the following code in the console and it will prevent you from disconnecting.
Ctrl+ Shift + i to open inspector view .

Then goto console and type this code:

      function ClickConnect(){
      console.log("Working"); 
      document.querySelector("colab-toolbar-button#connect").click() 
      }
      setInterval(ClickConnect,60000)


It would keep on clicking the page and prevent it from disconnecting.


### **Code**

In [0]:
### download SQUAD dataset on colab (found in 'Files/data' section)

#training set 
!wget -qq https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -P ./data
  
#dev set
!wget -qq https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -P ./data
  

In [0]:
def AQQAD_generator():
  
  #### dictionary dataset declartion 
  # Will be used to store final JSON format of the dataset
  AAQAD_dic = {}
  AAQAD_dic["version"] = "v1.0"
  AAQAD_dic['data'] = []
  Question_ID = 1
  min_num_questions = 3
  
  #### Imitate training & dev set
  with open('./data/train-v2.0.json') as json_file:  
    file = json.load(json_file)
    
  ## loop on each article...
  en_article_count = -1
  ar_article_count = 0
  for article in file["data"][295:300]:
    en_article_count += 1
#     ar_article_count += 1 
    print("##################################################")
    print("Article no ",en_article_count, " : ",article["title"])
        
    #extract all arabic paragraphs(if exists) from article title
    arabic_page_exist, article_ar_title, article_ar_paragraphs = get_arabic_paragraphs(article["title"])

    if arabic_page_exist:
      print("Arabic page exits !")
      article_paragraphs = []
      valid_article = False
      article_initiazlized = False
      
      ## send all english paragraphs along with the arabic ones 
      #  to be compared and to retrieve similarities between them...
      print("Collecting English paragraphs...")
      article_en_paragraphs = []
      for parag in article["paragraphs"]:
        article_en_paragraphs.append(parag["context"])      
      
      #find similarity between en_parags & wiki_ar parags
      print("Finding Similar Arabic paragraphs...")
      similar_parags_dic = get_similar_ar_paragraphs(article_en_paragraphs,article_ar_paragraphs)
      
      # if there is matching paragraphs ,iterate on them to
      # retrieve their Questions and Answers
      if len(similar_parags_dic) > 0:
        
#         # initiazlize article
#         if article_initiazlized == False :
#             AAQAD_dic['data'].append({'title' : article_ar_title,
#                                       'paragraphs': []})
#             article_initiazlized = True
        
        ##NOTE: article["paragraphs"][parag index] -> parag
        
        ## loop on questions for the each matched paragraph
        # key: parag index - val: corresponding ar paragraph
        print("Looping on ",len(similar_parags_dic)," matched paragraphs...")
        for key,val in similar_parags_dic.items():
          print("\nParagraph no: ",key," was found in arabic")
          print("\nQuestions:")
          ques_count = -1
          
          # boolean to check for at least 1 valid question for the current paragraph
          valid_questions = False
          qas = []
          
          for ques in article["paragraphs"][key]["qas"]:
            ques_count += 1
            print("\nquestion no: ",ques_count)

            # translate question to arabic
            translated_ques = translate_to_arabic(ques["question"])

            # find if first answer(or plausible answer) exist in the arabic matched paragraph
            print("Cheking Answer")

            if ques["is_impossible"] == False:
              translated_ans = translate_to_arabic(ques["answers"][0]["text"])
              en_ans = ques["answers"][0]["text"]
            else:
              translated_ans = translate_to_arabic(ques["plausible_answers"][0]["text"])
              en_ans = ques["plausible_answers"][0]["text"]

            #search for the existance of the correct answer in ar paragraph
            ans_exist, correct_ans, ans_index_start = find_answer(val,translated_ques,translated_ans,en_ans)
            if ans_exist:
              print("Answer exists for Question no: ",ques_count)
              if valid_questions == False:
                valid_questions = True

              # adding valid answer to appropiate dictionaries
              ans_details = {'text': correct_ans,
                            'answer_start':ans_index_start}
              if ques["is_impossible"] == False:
                qas.append({'question' : translated_ques,
                            'id': Question_ID,
                            'answers': [ans_details],
                            'is_impossible': False})                  
              else:
                qas.append({'plausible_answers':[ans_details],
                            'question' : translated_ques,
                            'id': Question_ID,
                            'answers': [],
                            'is_impossible': True})
              Question_ID += 1  
                     
            else:
              print("Answer do not exist for Question no: ",ques_count)
          
          #check for valid questions & minimum # of allowed questions to approve current paragraph
          if (valid_questions == True) & (len(qas) >= min_num_questions):
            valid_article = True            
            article_paragraphs.append({'qas': qas, 'context' : val })
#             article_paragraphs.append({'qas': qas, 'context' : val, 'context_en' : article["paragraphs"][key]["context"], 'similarity' : sim.similarity(val,translate_to_arabic( article["paragraphs"][key]["context"])) })
          else:
            # handling Question_ID issue
            Question_ID = Question_ID - len(qas) + 1
      
     ## add all paragraphs to the correct article in AAQAD_dic
      if valid_article == True:
            
        # initiazlize article once
        if article_initiazlized == False :
          AAQAD_dic['data'].append({'title' : article_ar_title,
                                        'paragraphs': []})
          article_initiazlized = True
          
        AAQAD_dic['data'][ar_article_count]['paragraphs'] = article_paragraphs
        ar_article_count += 1
        # Write current valid article To JSON file
        save_JSON(AAQAD_dic,ar_article_count)        
        # delay for 2 seconds between each article
        time.sleep(2)      
        
    else:
      print("Arabic page do NOT exist in Arabic")
#       ar_article_count -= 1
      
    
    print("####################################################")
  
  #Downloading lastest updated version
  save_JSON(AAQAD_dic,ar_article_count)
#   files.download('/content/data/AAQAD-v1.0.json')
#   print("\n\nLastest version downloaded !")
  
  return

### **Testing**

In [0]:
#testing 

AQQAD_generator()


##################################################
Article no  0  :  Asthma
Arabic page exits !
Finding Similar Arabic paragraphs...


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Looping on  25  matched paragraphs...

Paragraph no:  0  was found in arabic

Questions:

question no:  0
Cheking Answer
Answer exists for Question no:  0

question no:  1
Cheking Answer
Answer exists for Question no:  1

question no:  2
Cheking Answer
Answer exists for Question no:  2

question no:  3
Cheking Answer
Answer exists for Question no:  3

question no:  4
Cheking Answer
Answer exists for Question no:  4

question no:  5
Cheking Answer
Answer exists for Question no:  5

question no:  6
Cheking Answer
Answer exists for Question no:  6

question no:  7
Cheking Answer
Answer exists for Question no:  7

question no:  8
Cheking Answer
Answer exists for Question no:  8

question no:  9
Cheking Answer
Answer exists for Question no:  9

Paragraph no:  1  was found in arabic

Questions:

question no:  0
Cheking Answer
Answer do not exist for Question no:  0

question no:  1
Cheking Answer
Answer do not exist for Question no:  1

question no:  2
Cheking Answer
Answer do not exist for 

In [0]:

str_list = ['من طرف الجمهور الفرنساوي المبني على أساس الحرية، والسر عسكر الكبير بونابارته أمير الجيوش الفرنساوية، نعرف أهالي مصر جميعهم أن من زمان مديد السناجق الذين يتسلطون في البلاد المصرية، يعاملون بالذل والاحتقار في حق الملة الفرنساوية، ويظلمون تجارها بأنواع البلص والتعدي، فحضرت الآن ساعة عقوبتهم، وحسرت من مدة عصور طويلة هذه الزمرة المماليك المجلوبين من جبال الأباظة والكرجستان يفسدوا في الأقاليم الحسان ما يوجد في كرة الأرض كلها، فأما رب العالمين القادر على كل شيء قد حتم في انقضاء دولتهم', ' ', '  ', '', '']

# str_list = list(filter(None, str_list))

# str_list = filter(bool, str_list) # fastest

str_list = filter(len, str_list)

print(len(list(str_list)))


count = 1

if (count != 0) & ((count % 10) == 0):
  print("here")

  
# from google.colab import files

# files.download('/content/data/AAQAD-v1.0.json')


# data = {}
# data['test'] = qas
# with open('./data/test.json', 'w') as outfile:
#     json.dump(data, outfile)


# p = ['a','3','fd']

# for e in p:
#   print (p.index(e))

# print(REQUESTS)
